1) Prepare Colab

Open https://colab.research.google.com
 and click File → New notebook.

Set runtime to GPU: Runtime → Change runtime type → Hardware accelerator → GPU → Save.

2) Install libraries


In [ ]:
# Colab cell 1 - install required libs
!pip -q install transformers accelerate huggingface_hub safetensors sentencepiece einops
# optional (needed for 8-bit quantization / bigger models)
!pip -q install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.7 MB/s eta 0:00:00


In [ ]:
# Colab cell 2 - check GPU
import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except:
        print("GPU detected (name unavailable).")
else:
    print("No GPU detected. For faster runs enable GPU in Runtime → Change runtime type.")

Torch: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4


In [ ]:
# Colab cell 2: secure login (hidden input)
from huggingface_hub import login
from getpass import getpass

token = getpass("Enter your Hugging Face token (press Enter to skip if you don't have one): ")
if token:
    try:
        login(token=token)
        print("Logged into Hugging Face.")
    except Exception as e:
        print("Login failed:", e)
else:
    print("No token provided — will only try public models.")


Enter your Hugging Face token (press Enter to skip if you don't have one): ··········
Logged into Hugging Face.


In [ ]:
# Colab cell 3: try loading preferred models and fallback
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# List of models to try in order (you can edit this list)
# Note: first model is gated (Llama 2) — requires token and a large GPU.
model_candidates = [
    ("meta-llama/Llama-2-7b-chat-hf", True),     # gated - token recommended
    ("tiiuae/falcon-7b-instruct", False),        # large, may or may not fit
    ("gpt2", False)                              # small, should always work
]

def try_load(model_name):
    """Attempt to load tokenizer + model. Returns (tokenizer, model)."""
    print(f"\nAttempting to load: {model_name}")
    # choose dtype / device
    use_cuda = torch.cuda.is_available()
    device_map = "auto" if use_cuda else None
    torch_dtype = torch.float16 if use_cuda else None

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        # low_cpu_mem_usage reduces peak CPU RAM during loading
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map=device_map,
            torch_dtype=torch_dtype,
            low_cpu_mem_usage=True
        )
        print(f"Loaded {model_name} successfully.")
        return tokenizer, model
    except Exception as e:
        print(f"Failed to load {model_name}: {type(e).__name__}: {e}")
        return None, None

# iterate candidates and pick first that loads
loaded = False
for model_name, gated in model_candidates:
    # skip gated model if user didn't provide a token (to avoid pointless failure)
    if gated and not token:
        print(f"Skipping gated model {model_name} because no token was provided.")
        continue

    tok, mod = try_load(model_name)
    if tok is not None:
        tokenizer, model = tok, mod
        active_model_name = model_name
        loaded = True
        break

if not loaded:
    raise RuntimeError("All model load attempts failed. Try a smaller model or upgrade runtime (Colab Pro).")

# move model to device if not already mapped
if not next(model.parameters()).is_cuda and torch.cuda.is_available():
    model.to("cuda")
    print("Moved model to CUDA device.")

print(f"\nUsing model: {active_model_name}")



Attempting to load: meta-llama/Llama-2-7b-chat-hf
Failed to load meta-llama/Llama-2-7b-chat-hf: OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-68c91f1c-27b143cb104595062d6b8ad6;6276365f-2feb-4b0f-8b76-85b8b50be0f8)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-2-7b-chat-hf to ask for access.

Attempting to load: tiiuae/falcon-7b-instruct


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Loaded tiiuae/falcon-7b-instruct successfully.

Using model: tiiuae/falcon-7b-instruct


In [ ]:
# Colab cell 4: generation helper
import torch

def generate(prompt, max_new_tokens=150, do_sample=True, top_p=0.9, temperature=0.8):
    model_device = next(model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(model_device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            top_p=top_p,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text

# Example usage:
prompt = "Write a short poem about the stars."
print("Prompt:", prompt)
print("\nGenerated text:\n")
print(generate(prompt, max_new_tokens=120))


Prompt: Write a short poem about the stars.

Generated text:

Write a short poem about the stars.
Above, the night sky glows,
A myriad of twinkling stars,
Each one unique, yet all in sync,
A cosmic dance, an endless spin.

Infinite darkness, boundless light,
The stars have stories to delight,
A galaxy of wonders, ever bright,
A cosmic canvas, painted with sight.
